## Objective: Add supplier data to main data table

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [4]:
df = pd.read_csv("data_tables/for_adding_sup_data.csv")

In [ ]:
del df["Unnamed: 0"]

In [5]:
# No. of suppliers per TA
num_sup = df.groupby("tube_assembly_id").supplier.nunique()

In [6]:
# Annual usage by supplier
au_sup = pd.pivot_table(df, index = "supplier", values = "annual_usage", aggfunc = "sum")

In [21]:
# Make df with supplier and TA-id
df_sup_raw = num_sup.to_frame()
df_sup_raw["tube_assembly_id"] = df_sup_raw.index
df_sup_raw.index = range(df_sup_raw.shape[0])
df_sup_raw = df_sup_raw.rename(columns = {"supplier": "num_supplier"})

In [22]:
df_sup_raw.head()

,num_supplier,tube_assembly_id
0,1,TA-00002
1,1,TA-00004
2,1,TA-00005
3,1,TA-00012
4,1,TA-00013


In [24]:
# Make df with au and supplier
df_au_raw = au_sup.to_frame()
df_au_raw["supplier"] = df_au_raw.index
df_au_raw.index = range(df_au_raw.shape[0])
df_au_raw = df_au_raw.rename(columns = {"annual_usage": "total_annual_usage"})

In [25]:
df_au_raw.head()

,total_annual_usage,supplier
0,4,S-0003
1,1,S-0004
2,8733,S-0005
3,300,S-0006
4,1551,S-0007


In [28]:
df["quote_date"] = df["quote_date"].astype("datetime64")

In [40]:
# Min and max dates supplier
dt_sup = pd.pivot_table(df, index = "supplier", values = "quote_date", aggfunc = "max")

In [44]:
# Make df with last date and supplier
df_dt_raw = dt_sup.to_frame()
df_dt_raw["supplier"] = df_dt_raw.index
df_dt_raw.index = range(df_dt_raw.shape[0])
df_dt_raw = df_dt_raw.rename(columns = {"quote_date": "last_date_in_data"})

In [45]:
df_dt_raw.head()

,last_date_in_data,supplier
0,2012-05-09,S-0003
1,2013-07-22,S-0004
2,2014-07-24,S-0005
3,2013-12-19,S-0006
4,2011-12-01,S-0007


In [49]:
df.shape

(29720, 123)

In [50]:
# Merge df and df_sup_raw
df1 = df.merge(df_sup_raw, how = "left", on = "tube_assembly_id")

In [51]:
df1.shape

(29720, 124)

In [52]:
# Merge df1 and df_au_raw
df2 = df1.merge(df_au_raw, how = "left", on = "supplier")

In [53]:
df2.shape

(29720, 125)

In [58]:
# Merge df2 and df_dt_raw
df3 = df2.merge(df_dt_raw, how = "left", on = "supplier")

In [59]:
df3.shape

(29720, 126)

In [81]:
df3["num_days_supplier_known"] = pd.to_timedelta(df3["last_date_in_data"] - df3["quote_date"])

In [82]:
df3["num_days_supp_relationship"] = [i.days for i in df3["num_days_supplier_known"]]

In [87]:
df3["num_days_supp_relationship"].min()

0

In [88]:
del df3["num_days_supplier_known"]
del df3["tube_assembly_id"]
del df3["supplier"]
del df3["quote_date"]
del df3["last_date_in_data"]

In [89]:
df3.to_csv("data_tables/supplier_cleaned_final.csv")

In [2]:
df3 = pd.read_csv("data_tables/supplier_cleaned_final.csv")

In [3]:
df3.columns

Index(['Unnamed: 0', 'annual_usage', 'cost', 'diameter', 'wall', 'length',
       'num_bends', 'bend_radius', 'num_boss', 'num_bracket',
       ...
       'ex_EF-022', 'ex_EF-023', 'ex_NONE', 'ea_form_N', 'ea_form_Y',
       'ex_form_N', 'ex_form_Y', 'num_supplier', 'total_annual_usage',
       'num_days_supp_relationship'],
      dtype='object', length=124)

In [4]:
df3.head()

,Unnamed: 0,annual_usage,cost,diameter,wall,length,num_bends,bend_radius,num_boss,num_bracket,...,ex_EF-022,ex_EF-023,ex_NONE,ea_form_N,ea_form_Y,ex_form_N,ex_form_Y,num_supplier,total_annual_usage,num_days_supp_relationship
0,0,0,21.905933,6.35,0.71,137.0,8,19.05,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1,352193,482
1,1,0,12.341214,6.35,0.71,137.0,8,19.05,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1,352193,482
2,2,0,6.601826,6.35,0.71,137.0,8,19.05,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1,352193,482
3,3,0,4.687770,6.35,0.71,137.0,8,19.05,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1,352193,482
4,4,0,3.541561,6.35,0.71,137.0,8,19.05,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1,352193,482


In [5]:
y = df3["cost"]
del df3["cost"]

In [6]:
X_train, X_test, y_train, y_test = train_test_split(df3, y, test_size = 0.3)

In [7]:
X_train.head()

,Unnamed: 0,annual_usage,diameter,wall,length,num_bends,bend_radius,num_boss,num_bracket,num_other,...,ex_EF-022,ex_EF-023,ex_NONE,ea_form_N,ea_form_Y,ex_form_N,ex_form_Y,num_supplier,total_annual_usage,num_days_supp_relationship
16187,16187,0,9.52,0.71,49.0,3,31.75,0,0,0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1,352193,518
18108,18108,90,25.40,1.65,17.0,1,50.80,0,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1,48474,443
17824,17824,4,9.52,0.89,21.0,2,19.05,0,0,0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1,352193,810
24601,24601,0,12.70,1.24,25.0,1,38.10,0,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1,352193,518
27099,27099,0,12.70,1.65,104.0,4,25.40,0,0,0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1,352193,518
